# 大模型编码demo学习

In [ ]:
print("Hello, World!")

## 一、 使用阿里云百炼调用大模型 key base-url model
注意：
1. 在jupyter中似乎无法读环境变量，因此key使用明文，正常应该使用环境变量
    

In [ ]:
import os
from openai import OpenAI


# 打印环境变量
print(os.getenv("DASHSCOPE_API_KEY"))


client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    #打印环境变量
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

completion = client.chat.completions.create(
    # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    model="qwen-plus",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "你是谁？"},
    ],
    # Qwen3模型通过enable_thinking参数控制思考过程（开源版默认True，商业版默认False）
    # 使用Qwen3开源版模型时，若未启用流式输出，请将下行取消注释，否则会报错
    # extra_body={"enable_thinking": False},
)
print(completion.model_dump_json())

## 二、 使用langchain调用大模型
注意：
1. 因为langchain升级很快，会有很多东西过时，学习教程中的导入包以及predict方法无法使用，需要时刻注意官网的更新

In [103]:
# langchain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
import os

api_key=os.getenv("DASHSCOPE_API_KEY")
base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"

llm = ChatOpenAI(
    model="qwen-plus",
    temperature=0,
    api_key=api_key, 
    base_url=base_url)

prompt = PromptTemplate.from_template("你是一个起名字大师，请模仿实例起三个{country}名字，比如男孩名字经常叫做{boy_name}，女孩名字经常叫做{girl_name}")
message = prompt.format(country="中国特色的", boy_name="高义", girl_name="白洁")

print(message)
response = llm.invoke(message)
print(response.content)

当然可以！我来为你模仿这种风格，起三个具有中国特色、富有寓意和美感的名字，分别适合男孩和女孩，并保留类似“高义”“白洁”这种名字的韵味：

---

### 一、男孩名字  
**1. 周正**  
- **含义解析**：“周”有周全、严谨之意，“正”代表正直、端正。整体寓意为人品行端正、处事稳重。
- **风格特点**：简洁大气，传统儒家风范。

**2. 李诚**  
- **含义解析**：“诚”代表诚实、诚信，是中华文化中极为重视的品德之一。
- **风格特点**：朴实无华，强调德行为先。

---

### 二、女孩名字  
**1. 王静**  
- **含义解析**：“静”意为安静、文静，体现了中国传统对女性温柔内敛气质的赞美。
- **风格特点**：温婉大方，常见但不俗气。

**2. 张慧**  
- **含义解析**：“慧”即聪慧、智慧，表达对才智与灵秀之气的期许。
- **风格特点**：知性优雅，兼具文化内涵。

---

如果你希望这些名字能更具地域特色、时代感或用于小说、角色设定，也可以告诉我更多背景，我可以进一步定制化创作。


## 三、标准输出格式
1. 可自定义输出解析器 继承BaseOutputParser
2. temperature最好为0，防止模型输出格式的变化
3. 用于对数据的格式清洗 todo 代码

In [ ]:
# todo 标准输出格式
# 1. 可自定义输出解析器 继承BaseOutputParser
# 2. temperature最好为0，防止模型输出格式的变化
# 3. 用于对数据的格式清洗 todo 代码


## 四、模型IO 大语言模型的交互接口
- prompts ： 模版化
- Language models ：LLM 和 chat model 
- Ourput parser ： 格式处理，比如处理为JSON和其他系统对接

### prompts 模版 提示词提炼为模版，实现提示词的复用、版本管理、动态变化

优秀提示词：
- 角色：引导AI进入具体场景，赋予其身份
- 问题：描述问题和困惑，以及背景信息
- 目标：描述需求，希望达成的目标
- 要求：告诉AI回答是注意什么，或者如何回复

比如 ：我想去旅游，有什么建议？可以拓展为：
我想旅游，请根据以下条件给我推荐几个目的地和行程安排：
- 出发地：[你的城市]
- 预计时间：[几天]，大致出发时间：[月份或节假日]
- 预算：[金额范围]
- 偏好：[自然风光 / 人文历史 / 海边度假 / 城市漫步 / 少人安静 等]
- 出行方式：[飞机 / 高铁 / 自驾 / 无限制]
- 同行人员：[一个人 / 情侣 / 家庭带小孩 / 老人 等]

请给出推荐的目的地、适合的时间、交通方式、住宿建议和注意事项。


### PromptTemplate 字符串模版，对应LLM

In [ ]:
from langchain.prompts import PromptTemplate
import os

prompt = PromptTemplate(
    input_variables=["country"],
    template="What is the capital of {country}?"
)

prompt.format_prompt(country="China")

### ChatPromptTemplate 对话模版具有结构，对应chat model
- 角色 system user ai
- 对话模式

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant,your name is {name}"),
    ("user", "hello,{name}"),
    ("ai", "hi"),
    ("user", "What is the capital of {country}?"),
    ("ai", "The capital of {country} is {capital}.")
])

prompt.format_messages(country="China", name="John", capital="Beijing")

In [ ]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

system_message = SystemMessage(content="You are a helpful assistant,your name is {name}",additional_kwargs={"name":"John"})
human_message = HumanMessage(content="hello,{name}",additional_kwargs={"name":"John"})
ai_message = AIMessage(content="hi",additional_kwargs={"name":"John"})
human_message = HumanMessage(content="What is the capital of {country}?",additional_kwargs={"country":"China"})
ai_message = AIMessage(content="The capital of {country} is {capital}.",additional_kwargs={"country":"China","capital":"Beijing"})

messages = [system_message,human_message,ai_message,human_message,ai_message]

print(messages)




In [ ]:
# ChatMessagePromptTemplate  role 自定义角色
from unittest import result
from langchain.prompts import AIMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate

prompt = "你真是个{subject}，{subject}真是个{subject}"

result = ChatMessagePromptTemplate.from_template(role="user",template=prompt) 

result.format(subject="猪猪")

In [ ]:
# AIMessagePromptTemplate 
from unittest import result
from langchain.prompts import AIMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate

prompt = "你真是个{subject}，{subject}真是个{subject}"

result = AIMessagePromptTemplate.from_template(template=prompt) 

result.format(subject="猪猪")

### 自定义模版


In [ ]:
## 函数大师 ： 根据函数名称寻找函数，并给出中文说明

from langchain.prompts import StringPromptTemplate

def hello_world():
    print("hello world")
    return "hello world"


PROMPT = """\
你是一个非常有经验的程序员，现在给你如下函数，你会严格按照如下格式，输出这段代码的函数名称、源代码、中文解释。
函数名称：{function_name}
源代码：{source_code}
中文解释：
"""

import inspect

def get_source_code(function_name):
    return inspect.getsource(function_name)

# 自定义模版class


class CustomPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        # 获取函数源代码
        source_code  = get_source_code(kwargs["function_name"])
        # 生成提示词模版
        prompt = PROMPT.format(function_name=kwargs["function_name"].__name__,source_code=source_code)
        return prompt
    
a = CustomPromptTemplate(input_variables=["function_name"])
pm = a.format(function_name=hello_world)

print(pm)

# 和LLM交互
from langchain_openai import ChatOpenAI
import os

api_key="sk-730784af62d14bc6b5067a2e6633f096"
base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"

llm = ChatOpenAI(
    model="qwen-plus",
    temperature=0,
    api_key=api_key, 
    base_url=base_url)

response = llm.invoke(pm)

print(response.content)





### jinja2 与f-string

In [ ]:
# f-string 格式化字符串 python内置的模版引擎
from langchain.prompts import PromptTemplate

f_string = """
请给我讲一个关于{topic}的故事，故事要包含{topic}的{detail}
"""

prompt = PromptTemplate.from_template(f_string)

print(prompt.format(topic="张三",detail="身世"))

In [ ]:
# jinjia2 模版引擎
! pip install jinja2

In [ ]:
# jinjia2 模版引擎 更为灵活生成各种标记格式的文档
from langchain.prompts import PromptTemplate

jinjia2_stirng = "请给我讲一个关于{{topic}}的故事，故事要包含{{topic}}的{{detail}}"

prompt = PromptTemplate.from_template(jinjia2_stirng,template_format="jinja2")

print(prompt.format(topic="张三",detail="身世"))

### 三层提示词设计 性格-行为-禁止
- 组合模版 
- 多层模版 
- Final prompt & Pipeline prompts


In [ ]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [ ]:
PROMPT = """
你是一个经验丰富的程序员，你是中国人，住在北京。
你总是穿格子衫。
你从不说脏话。
"""



In [ ]:

from langchain.prompts import PromptTemplate

# 子模板定义
character_prompt = PromptTemplate.from_template("你是一个经验丰富的{person}，你是{country}人，住在{city}。")
behavior_prompt = PromptTemplate.from_template("你总是穿{clothes}。")
forbidden_prompt = PromptTemplate.from_template("你从不{forbidden}。")

# 最终模板
final_prompt = PromptTemplate.from_template("""
{character}
{behavior}
{forbidden}
""")

# 模板列表
pipeline_prompts = [
    ("character", character_prompt),
    ("behavior", behavior_prompt),
    ("forbidden", forbidden_prompt)
]

# 输入参数
input_values = {
    "person": "导游",
    "country": "中国",
    "city": "成都",
    "clothes": "唐装",
    "forbidden": "说脏话"
}

# 用于保存每一层输出
intermediate_values = {}

# 使用循环逐步构建中间变量
for name, prompt in pipeline_prompts:
    intermediate_values[name] = prompt.format(**input_values)

print(intermediate_values)

# 最终生成完整提示
final_result = final_prompt.format(**intermediate_values)

# 输出
print(final_result)






### 示例选择器 


#### 根据长度自动选择



In [9]:
# 根据输入的提示词长度综合计算最终长度 智能截取或者添加提示词

from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector



# 假设已经有的示例词组
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "高兴", "output": "难过"},
]

# 构造提示词模版
prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词：{input} \n-> 反义词：{output}"
)



# 构造长度示例选择器
example_selector = LengthBasedExampleSelector(
    # 提示词示例组
    examples=examples,
    # 提示词模版
    example_prompt=prompt,
    # 最大长度
    max_length=17,
    # get_text_length
)
# 动态构造提示词
dynamic_prompt = FewShotPromptTemplate(
    # 示例选择器
    example_selector=example_selector,
    # 提示词模版
    example_prompt=prompt,
    # 提示词
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective} \n-> 反义词：",
    input_variables=["adjective"],
)




In [ ]:
print(dynamic_prompt.format(adjective=" big "))

In [ ]:
# 太长，会自动截取
print(dynamic_prompt.format(adjective=" big big big big big big ig big big big big big big big big big"))

#### 根据输入相似度选择示例（最大边际相关性）

- MMR 信息检索方法，目标是在相关性和多样性之间找到平衡
- 余弦相似度最大 余弦相似度 点积/模长相乘
- 对过于相似的 迭代过程惩罚
- 


In [2]:
# MMR示例
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate

# 假设已经有的示例词组
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "sunny", "output": "gloomy"},
]

/Users/zhangzhongyuan/PycharmProjects/LLM-learn/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
! pip install tiktoken
! pip install faiss-cpu
! pip install langchain-community
! pip install dashscope

In [ ]:
! pip install -U openai

#### 直接调用阿里云示例

In [28]:
from openai import OpenAI

import os

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),  # 如果您没有配置环境变量，请在此处用您的API Key进行替换
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"  # 百炼服务的base_url
)

completion = client.embeddings.create(
    model="text-embedding-v4",
    input='衣服的质量杠杠的，很漂亮，不枉我等了这么久啊，喜欢，以后还来这里买',
    dimensions=1024, # 指定向量维度（仅 text-embedding-v3及 text-embedding-v4支持该参数）
    encoding_format="float"
)
print(completion.model_dump_json())

{"data":[{"embedding":[0.00954423751682043,-0.11166470497846603,0.0002610872033983469,-0.04448245093226433,0.018730206415057182,-0.013019428588449955,0.015362495556473732,-0.0032817272003740072,-0.0013390234671533108,0.12496358901262283,0.04709063470363617,-0.03614199161529541,0.027127988636493683,-0.03147018700838089,0.0708509162068367,-0.07320114970207214,-0.038635529577732086,-0.08036649227142334,-0.04929756000638008,-0.03321853280067444,0.023817600682377815,0.05574636906385422,0.028919324278831482,-0.0031061763875186443,-0.027844524011015892,0.03485222905874252,-0.01501855906099081,-0.05649156495928764,-0.008290302008390427,0.06265375763177872,-0.024706102907657623,0.0066888476721942425,-0.042590800672769547,-0.00920030102133751,0.04737725108861923,0.019876662641763687,-0.016265328973531723,0.029979795217514038,0.005911407992243767,0.006860815919935703,0.005273692775517702,-0.019733354449272156,0.0269416905939579,0.0219976045191288,-0.0203352440148592,0.024433819577097893,-0.034938

#### langchain 调用embedding

In [44]:
from langchain.embeddings import DashScopeEmbeddings
import os


embeddings = DashScopeEmbeddings( 
    model="text-embedding-v4",
)
print(embeddings.embed_query("衣服的质量杠杠的，很漂亮，不枉我等了这么久啊，喜欢，以后还来这里买"))

[0.0054546226747334, -0.11254218965768814, -0.01061803288757801, -0.040052834898233414, 0.023864440619945526, -0.024043647572398186, 0.008960365317761898, -0.001020174939185381, 0.0012936528073623776, 0.12030784040689468, 0.033750709146261215, -0.0359310656785965, 0.02667202055454254, -0.0334818996489048, 0.08255483210086823, -0.06869613379240036, -0.05274668335914612, -0.085780568420887, -0.04934174567461014, -0.02753818966448307, 0.017651919275522232, 0.0448615625500679, 0.04029177501797676, -0.007213094271719456, -0.03873864561319351, 0.044652488082647324, -0.011095919646322727, -0.03960481658577919, -0.013492817059159279, 0.06290176510810852, -0.029539337381720543, 0.0070637548342347145, -0.019429059699177742, -0.00682854512706399, 0.031570352613925934, 0.020534170791506767, -0.024835146963596344, 0.029121188446879387, -0.003991096280515194, 0.004114300943911076, -0.002232624450698495, -0.020414698868989944, 0.02499941922724247, 0.014194712042808533, -0.02108672633767128, 0.0268960

In [46]:
# 调用MMR

# 模版
prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词：{input} \n-> 反义词：{output}"
)

# 示例词组
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "高兴", "output": "悲伤"},
]

# MMR选择器
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # 示例词组
    examples,
    # 嵌入模型
    DashScopeEmbeddings(model="text-embedding-v4",),
    # 向量库
    FAISS,
    # 选择数量
    k=2,
)

# 小样本模版
mmr_prompt = FewShotPromptTemplate(
    # 示例选择器
    example_selector=example_selector,
    # 提示词模版
    example_prompt=prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective} \n-> 反义词：",
    input_variables=["adjective"],
)

In [ ]:
# 输入情绪的，应该返回情绪的
print(mmr_prompt.format(adjective="worried"))

In [47]:
# 输入颜色，返回颜色
print(mmr_prompt.format(adjective="难过"))

给出每个输入词的反义词

原词：高兴 
-> 反义词：悲伤

原词：energetic 
-> 反义词：lethargic

原词：难过 
-> 反义词：


In [ ]:
! pip install langchain-chatchat -U

In [50]:
! pip install chromadb==0.4.15

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/


In [49]:
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import DashScopeEmbeddings

prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词：{input} \n-> 反义词：{output}"
)

# 示例词组
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "高兴", "output": "悲伤"},
]

# MMR选择器
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # 示例词组
    examples,
    # 嵌入模型
    DashScopeEmbeddings(model="text-embedding-v4",),
    # 向量库
    Chroma,
    # 选择数量
    k=2,
)

# 小样本模版
mmr_prompt = FewShotPromptTemplate(
    # 示例选择器
    example_selector=example_selector,
    # 提示词模版
    example_prompt=prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective} \n-> 反义词：",
    input_variables=["adjective"],
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
